In [1]:
from train_model import train_model
from model import ShakespearModel

N_EPOCHS = 2
N_TOKENS = 250  # N
N_LAYERS = 6  # L
N_HEADS = 4  # h
N_WORKERS = 2
BATCH_SIZE = 5  # B
D_MODEL = 200  # d
D_K = 32
D_V = D_K
D_FF = 512
RAW_DATA_PATH = './datasets/shakespear_corpus.txt'

#trained_mod = train_model(N_EPOCHS, N_TOKENS, N_LAYERS, N_HEADS, BATCH_SIZE, D_MODEL, D_K, D_V, D_FF)

In [2]:
from train_model import load_data
from parsing.CharDataSet import CharDataSet
from torch.utils.data.dataloader import DataLoader

raw_data = load_data(RAW_DATA_PATH)

tokenized_data = CharDataSet(N_TOKENS, raw_data)
data_loader = DataLoader(
    tokenized_data,
    shuffle=False,
    batch_size=BATCH_SIZE,
    num_workers=N_WORKERS,
)

trained_mod = ShakespearModel(N_LAYERS, N_HEADS, D_MODEL, D_FF, D_K, D_V, BATCH_SIZE, N_TOKENS, tokenized_data.get_vocab_size())
print(trained_mod.param_norm())

tensor(134.9826)


In [3]:
seed = "O God, O God!"
idx = tokenized_data.encode(seed)

In [4]:
print(trained_mod.param_norm())
new_tokens = trained_mod.generate(idx, n_new_tokens=500)
print(trained_mod.param_norm())
print(new_tokens)

tensor(134.9826)
idx tensor([[27.,  1., 19., 53., 42.,  6.,  1., 27.,  1., 19., 53., 42.,  2.]])


In [8]:
print(tokenized_data.decode(new_tokens))
# ElllEsEsEEllEllElCpElCllllVellllllllEVVglellVllell
# EEEEEsEkEEEEllElEElllCllllellellllelleeglllllllllg
# lllElglElElllEllElElEeElllleellllllElgggelglglllll

 God, O God!HXY   Y   x Y &  H   Ox&WW   xx xx XHWiYY   xxx   YXXxxxHx x XHHHY yXxxxx xYH H  HYHH ?H H XxYXx x         Y YY xXW    xWW W       :Yx   P     g XH  WYYjt    X Yx :?  x:xg xxxbjxxx   xYYx Wxj jxW&&&H& XX OxW  xWxWW    gOYO x  xxxxxx xg? Sj j  Yx     YnxYY   WYxx xYY Yx x:xx x x x xUx x YYY  xY x     bg xYY XxY  x:YWx Xxx   &xx xxxXxH  HYX xx &xY    Y x   WWWxx       b   W    : gOW   ,xxx   YW  Y jWWjg  j X   ?Wx:   xggx lxgW x   :jYxxxxxjxxjxYxx xbajX   OPXWx axx xj x xW  YWxYxx  x Y


In [6]:
for name, param in trained_mod.named_parameters():
    print(name, param.size())

WTE.weight torch.Size([65, 200])
transformer.blocks.0.CausalSelfAttn.to_Q.weight torch.Size([128, 200])
transformer.blocks.0.CausalSelfAttn.to_Q.bias torch.Size([128])
transformer.blocks.0.CausalSelfAttn.to_K.weight torch.Size([128, 200])
transformer.blocks.0.CausalSelfAttn.to_K.bias torch.Size([128])
transformer.blocks.0.CausalSelfAttn.to_V.weight torch.Size([128, 200])
transformer.blocks.0.CausalSelfAttn.to_V.bias torch.Size([128])
transformer.blocks.0.CausalSelfAttn.W_O.weight torch.Size([200, 128])
transformer.blocks.0.CausalSelfAttn.W_O.bias torch.Size([200])
transformer.blocks.0.LayerNorm_1.weight torch.Size([200])
transformer.blocks.0.LayerNorm_1.bias torch.Size([200])
transformer.blocks.0.FFN.L1.weight torch.Size([512, 200])
transformer.blocks.0.FFN.L1.bias torch.Size([512])
transformer.blocks.0.FFN.L2.weight torch.Size([200, 512])
transformer.blocks.0.FFN.L2.bias torch.Size([200])
transformer.blocks.0.LayerNorm_2.weight torch.Size([200])
transformer.blocks.0.LayerNorm_2.bias t

In [ ]:
for name, param in trained_mod.transformer.blocks[0].CausalSelfAttn.named_parameters():
    print(name, param)